# 유형1 : 데이터 전처리 작업 3문제 * 10 [30점]

In [ ]:
# 소수점 아래 2째자리까지 표시되도록 설정하기
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

### 1. 데이터 불러오기

In [ ]:
import pandas as pd

# csv 파일 불러오기 
df = pd.read_csv('파일경로.csv')
## 불러오면서 인덱스 컬럼 설정
df_csv1 = pd.read_csv('파일경로.csv', index_col='인덱스로 삼을 칼럼명')
## 불러오면서 구분자 지정
df_csv2 = pd.read_csv('파일경로.csv', sep='\t')
## 불러올 때 언어 설정
df_csv3 = pd.read_csv('파일경로.csv', encoding='euc-kr')  # 한글지원 : euc-kr, cp949
## 불러올 때 결측치 필터 사용 끄기
df_csv2 = pd.read_csv('파일경로.csv', na_filter=False)


# excel 파일 불러오기 (엑셀은 엔진 설정해주기!!)
df_excel = pd.read_excel('파일경로.xlsx', engine='openpyxl')

# csv 파일 저장하기
df.to_csv('경로.csv', index=False)
# excel 파일 저장하기
df.to_excel('경로.xlsx', index=False)

### 2. 데이터 구조 확인 

In [ ]:
# 데이타가 많은 경우 모두 출력 안되고 ... 으로 생략해서 출력됩니다.
# 생략되지 않는 행, 열의 개수를 설정하여 생략되지 않고 출력되도록 합니다.
pd.set_option('display.max_rows', 800)    #출력할 max row를 지정
pd.set_option('display.max_columns', 100)  #출력할 max columns를 지정

In [ ]:
# 처음부터 n개 행의 데이터 확인
df.head()
# 끝부터 n개 행의 데이터 확인
df.tail()

# 데이터프레임 row개수, colum개수, Not null,dtype 등 정보 확인
df.info(memory_usage='deep')
# 데이터프레임 통계 정보 확인
df.describe()

# 데이터프레임의 행, 열의 수를 (행, 열)형태의 튜블로 반환
df.shape
# 데이터프레임의 (행 x 열)의 전체 데이터 수를 반환
df.size
# 데이터프레임 데이터 타입 확인
df.dtypes

# 시리즈
sri = df['특정 컬럼']
# 시리즈 데이터 타입 확인
sri.dtypes

#데이터프레임의 인덱스 확인 (보통 0으로 시작해서 몇으로 끝나고 스텝이 몇인지 등을 보여줌)
df.index
# 데이터프레임의 컬럼 확인
df.columns          # 타입 index
df.columns.values   # 타입 array
# 데이터프레임의 구성요소 2차원으로 보기
df.values           # 타입 array

# 시리즈의 구성요소
sri.values          # 타입 array 해당 시리즈 값 배열로 쭉 나옴

In [ ]:
# 시리즈 중 컬럼에 존재하는 값 종류만 확인(결측치 포함, ndrarray로 반환)
sri.unique()       
# 시리즈 중 컬럼에 존재하는 값 종류의 개수만 확인
sri.nunique()
# 시리즈 중 컬럼에 대해 값 별 개수 확인(결측치 미포함, Series 반환)
sri.value_counts() 

DataFrame의 index, columns 및 Series의 index 는 대입연산을 사용하여 변경 가능 다만, 개수가 동일해야 함  
  
value는 인덱스를 이용해서 변경 가능  

In [ ]:
# 컬럼 개수 확인 -> 컬럼 이름 리스트를 대입 연산자로 넣기(단, 개수 동일하게)
df.columns = ['컬럼1', '컬럼2', '컬럼3']

### 3. 데이터 타입 변경
데이터 타입 변경 전 데이터 조작이 필요할 수 있음 (예) 불필요 문자/콤마/공백 제거, 단위 변환 등

In [ ]:
# 시리즈 데이터 타입 변경 ('int', 'int32', 'int64', 'float', 'str', 'category' 등)
## 넘파이 타입으로 하려면 넘파이 임포트 하여야함 (np.int16, np.float32, np.datetime64 등)
sri.astype('타입')

# 원하는 자료형으로 바꾸지 못하는 장애요소 제거하고 변환
## replace 사용 시 regex=True 옵션을 사용하면 일부 내용만 변경대상으로 지정할 수 있음
## 여기서 regex는 '정규표현식'을 의미
sri.replace('변경전', '변경후', regex=True)
# 변경할 내용이 두가지 이상일 때 list나 dic을 사용하여 변환
list1 = ['변경전1', '변경전2']
list2 = ['변경후1', '변경후2']
sri.replace(list1, list2, regex=True)
dic = {'변경전1':'변경후1', '변경전2':'변경후2'}
sri.replace(dic, regex=True).astype('int64')

## ,콤마는 메타 문자이므로 제거하고자 하는 경우 \, 처럼 역슬레시 사용
## 메타문자 종류 , . + ? * ^ $ 등
sri.replace(['\메타문자', '변견전2'], '', regex=True)  # 변경후 모습이 같다면 하나만 적어도 됨

Series는 Accessor라는 것을 가지고 있다. Accessor를 사용하기 위해서라도 데이터 타입 변경 필요
- dt : Datetime, Timedelta, Period
- str : String
- cat : Categorical 

In [ ]:
#Accessor 사용 .str accessor 사용하면 각 행에 문자열 처럼 접근 
sri.str[1:-1].astype('category')

일반적인 np.datetime64 나 category 타입 변경은 astype으로 변경 가능, 특수한 경우는 아래 방법 사용

In [ ]:
# datetime
import numpy as np
sri.astype(np.datetime64)  # 일/월/년 순으로 보기도 함 이런 경우 to_datatime을 사용
# 내가 원하는 방식으로 날짜를 읽어오고 싶을 때
## %Y: 4글자 년도, %y : 2글자 년도, %m : 2글자 월, %d : 2글자 일
pd.to_datetime(sri, format='%y-%m-%d')

# category
sri.astype('category')  # 이렇게 변환한 자료에 sort_values 적용하면 가나다 순 정렬됨 
# 카테고리 순서를 직접 지정하고 싶을 때
temp = pd.Categorical(sri, categories=["범주1", "범주2"], ordered=True)  # 이건 지금 시리즈는 아님
sri = temp  # 이건 시리즈
sri.sort_values() # 원하는 순서대로 정렬 할 수 있음

### 4. 데이터 정렬

In [ ]:
# sort_index() : 인덱스 정렬
# [3-24] '측정일시'를 index로 설정하고,
# index 기준으로 오름차순 정렬해서 df1으로 이름 붙입니다.
# 그래프에서 y축으로 사용하려고 합니다.
df1 = df.set_index('측정일시').sort_index()

# sort_values() : 밸류 정렬, 디폴트(내림차순)
df.sort_values('정렬기준칼럼', ascending=False)
# 오름차순인 경우
df.sort_values('정렬기준칼럼', ascending=True)
# 정렬 기준이 여러개인 경우
df.sort_values(['1차기준', '2차기준'], ascending=[False, False])  
# 정렬 기준이 여러개, 내림차순 오름차순도 여러개인 경우
df.sort_values(['1차기준', '2차기준'], ascending=[True, False])

### 5. 데이터 통계 확인

#### (1) 통계 함수
- axis=0 : 기본 값으로 행을 이동하면서, 행과 행의 연산을 수행한다.(수직으로 연산)
- axis=1 : 컬럼을 이동하며 컬럼과 컬럼의 연산을 수행한다.(수평으로 연산)

In [ ]:
# 기초 통계량 확인
df.describe()

# 개수 세기
df.count()
# 합계
df.sum()
# 평균
df.mean()
# 표준편차 
df.std()
# 분산
df.var()
# 중앙값
df.median()
# 최빈값
df.mode()  # 시리즈로 나옴
# 최빈값 1개
df.mode()[0]
# 최대값
df.max()   # 최대값, 최소값은 문자열에 대해서도 반응하는데 이는 ord(문자) 코드 숫자 기반
# 최소값
df.min()

# 분위수
df.quantile([0.25, 0.5, 0.75])    # 1사분위 0.25, 2사분위 0.5, 3사분위 0.75, 
# IQR
df.quantile(0.75) - sri.quantile(0.25)


In [ ]:
# 통계 함수 여러개 적용하고 싶을 때 1
df.apply(["min", "max", "mean"])    # apply 사용하면 어떤 함수든 적용 가능
# 통계 함수 여러개 적용하고 싶을 때 2
df.agg(["min", "max", "mean"])

#### (2) 그룹별 통계
- df.groupby(그룹명).통계함수() : 적용가능한 모든 단위
- df.groupby(그룹명)[칼럼].통계함수 : Series 단위
- df.groupby(그룹명)[[[칼럼1, 칼럼2 ... ]].통계함수 : 특정 컬럼 단위  
- 그룹별로 통계치 구할 땐 agg(['var', 'std', 'mode'])로 여러개 가능

* 통계 함수는 numerical 값에 적용되기 때문에 object는 형변환이 필요함

In [ ]:
# [2-54] 대륙별 주류 소비량 중앙값을 계산해 봅니다.
df.groupby('대륙').median()

# [2-55] 대륙별 맥주 소비량 평균은?
df.groupby('대륙')['맥주'].mean()

# [2-56] 전세계 맥주 소비량 평균보다 많은 맥주를 소비하는 대륙은?
temp = df.groupby('대륙')[['맥주']].mean()
temp[temp['맥주']> df['맥주'].mean()]

# 그룹바이 기준이 두개 이상인 경우 멀티 인덱스로 작업
# [3-34] df_dust에서 '년', '월'별 '미세먼지(㎍/㎥)' 데이터의 평균을 구해
# DataFrame으로 만들어 meandf 라는 이름을 지정합니다.ㅣ
meandf = df_dust.groupby(['년', '월'])[['미세먼지(㎍/㎥)']].mean()
meandf.head()

# [3-35] meandf에서 2017년 6월까지의 데이터만 출력합니다.  
meandf.loc[:(2017,6),:] # 멀티인덱스 인경우 튜플로 표시한다.

#### 피벗 테이블 : 행, 열 모두에 그룹으 지정하여 통계값 구하기
- df.pivo_table(index=행방향칼럼, columns=열방향칼럼, values=집계대상칼럼, aggfunc=통계함수)
- index, columns는 범주형, values는 연속형 사용

In [ ]:
# [2-58] pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구합니다.
df.pivot_table(index='대륙', values=['맥주', '와인'], aggfunc=['mean', 'median', 'max'])

# [2-59] pivot_table을 사용하여 대륙별(columns), '맥주'와 '와인'의 mean, median값을 구합니다.
df.pivot_table(columns='대륙', values=['맥주', '와인'], aggfunc=['mean', 'median', 'max'])

# [2-60] groupby를 사용하여 대륙별, '맥주'와 '와인'의 mean, median, max 값을 구합니다.
df.groupby('대륙')['맥주', '와인'].agg(['mean', 'median', 'max'])
# [2-58] 과 비슷하지만 피벗테이블을 쓰느냐 그룹바이를 쓰느냐에 따라 데이터프레임 구조가 미묘하게 다르다.

#### (3) Index, Columns 상호변경


- Columns to Index : df.set_index(['인덱스로 이동시킬 컬럼 명'..])
- Index to Columns : df.reset_index()

In [ ]:
# [2-32] 국가별 주류 소비량 합계(맥주, 증류주, 와인의 합)를 구해 봅니다.
df.set_index('국가')[['맥주', '증류주', '와인']].sum(axis=1)

# [2-33] df를 ['대륙', '국가']를 index로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장합니다.
df = df.set_index(['대륙','국가']).sort_index()

# [2-34] df의 index를 모두 columns로 이동합니다.
df = df.reset_index()
df.head()

### 7. 데이터 정제하기

#### 1) 결측치
- nan : 넘파이 배열에서 결측치 나타내는 경우
- NaN : 판다스에서 시리즈나 데이터프레임에서 결측치 나타내는 경우 

In [ ]:
# 결측치 찾기
df.isna()
df.isnull()
# 결측치 합계 구하기
df.isna().sum()

# 결측치가 아닌 것 찾기
df.notna()
df.notnull()

# 불리언 인덱싱으로 결측치만 찾기
df[df['컬럼'].isna()]

- 범주형 데이터 : 다른 범주로 만들어 채우기
- 연속형 데이터 : 0으로 채우기, 평균값으로 채우기, 범주별 평균값으로 채우기

In [ ]:
# 데이터 프레임의 결측치 채우기 (모두 같은 값으로)
df = df.fillna('대체값')

# 특정한 하나의 컬럼 결측치 채우기
df['칼럼'] = df['칼럼'].fillna('대체값')
df.loc[df['칼럼'].isna(), '대륙'] = '대체값'

DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
- 결측치 제거에 사용되는 메서드
- how='any' : 결측치가 하나라도 포함된 행 삭제
- how='all' : 모든 데이터가 결측치인 행 삭제
- axis=1 : 컬럼에 대해 동작
- thresh=숫자 : 숫자 이상의 데이터를 가진 행은 삭제 안함
- subset=[컬럼이름1, ...] : subset으로 지정된 컬럼만 사용하여 삭제 대상 검색

#### 결측치 제거

In [2]:
# 결측치 제거
df.dropna()

# [3-30] df_dust 에서 ['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)']에서
# 모든 데이터가 결측치인 행을 제거하여 결과를 temp1으로 저장합니다
temp1 = df_dust.dropna(how='all', axis=0, subset=['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)'])

# [3-32] df_dust 에서 ['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)']에서
# 2개 이상의 데이터를 가진 행은 제거하지 않은 결과를 temp3로 저장합니다.
# (= 3개의 정보 중 1개의 데이터만 가진 행을 제거함)
temp3 = df_dust.dropna(thresh=2, axis=0, subset=['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)'])

#### 결측치 채우기
Series.mask(조건, 조건이 참일 때 사용할 값 또는 값 목록)
- 조건이 True인 것에 대해 다른 값을 변경합니다.
- sri.isna() : NA값에 대해 True, NA아닌 것은 False

Series.where(조건, 조건이 거짓일 때 사용할 값 또는 값 목록)
- 조건이 False인 것에 대해서 다른 값으로 변경합니다.
- sri.notna() : NA값에 대해 False, NA아닌 것은 True

In [ ]:
# temp의 'A' 열에 대해서 결측치인 경우 'B'의 값으로 대체합니다.
temp['A'].mask(temp['A'].isna(), temp['B'])
# temp의 'A' 열에 대해서 결측치인 경우 'C'의 값으로 대체합니다.
temp['A'].where(temp['A'].notna(), temp['C'])

#### 2) 이상치

### 3) 파생 변수

#### (1) 포함된 목록 확인
- sri.isin() : 
- '찾을내용' in str : 

In [ ]:
df[df['category'].isin(['TV/방송', '게임'])]  # 또는 조건 isin으로도 해결 가능

# isin([]) 목록이 한개여도 리스트 타입으로 넣어줘야 함
df.loc[df['category'].isin(['음악/댄스/가수'])].sort_values('subscriber', ascending=False).head()

#### (2) str Accessor 사용
- str.contains('문자열') : 특정 문자열을 포함하는지 아닌지를 True/False로 반환 (Boolean Indexing 조건으로 사용 가능)
- str.upper() : 영문자 소문자를 대문자로 변경  /  str.lower() : 영문자 대문자를 소문자로 변경  
- 세부 내용 : https://pandas.pydata.org/docs/reference/series.html#string-handling

Series의 데이터를 list 및 ndarray로 반환 (데이터프레임 말고 시리즈에서 변환하는 것)
- Series.to_list() : Series의 values를 list로 반환
- Series.to_numpy() : Series의 values를 ndarray로 반환 (Series.values 와 동일)

In [ ]:
# [1-44] title에 'KBS'가 포함된 채널 명 목록을 만들어 봅니다.
df[df['title'].str.contains('KBS')]
# 대소문자 구분 없이 검색 하려면? !str 두번 쓰면 됨
df[df['title'].str.lower().str.contains('kbs')]

# Series 형태를 list나 array 형태로 변환하기
df[df['title'].str.contains('KBS')]['title'].to_list()
df[df['title'].str.contains('KBS')]['title'].to_numpy()

In [ ]:
# datetime에서 년 정보만 가져오기
df['측정일시2'].dt.year
# datetime에서 월 정보만 가져오기
df['측정일시2'].dt.month
# datetime에서 일 정보만 가져오기
df['측정일시2'].dt.day
# 월요일이 0 일요일이 6
df['측정일시2'].dt.dayofweek

#### (3) 불리언 인덱싱
- df.loc[조건] : 조건은 boolean dtype이어야함 (기호 : | 선언, & 연언, ~ 부정) !!참 거짓으로 나올 수 있어야함
- df.iloc[행범위, 열범위] : 인덱스 번호로 인덱싱 혹은 슬라이싱

In [ ]:
# [예시 1-36] 'category'가 '음악/댄스/가수'인 채널의 subscriber TOP5를 알아봅니다.
df.loc[df['category']=='음악/댄스/가수'].sort_values('subscriber').head()

# [2-23] 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬해 보자
df.loc[(df['와인']) > (df['맥주']+df['증류주'])].sort_values('대륙')

# [2-24] 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색해 보자
df.loc[(df['맥주']>230) & (df['와인']>230)]['국가']

# [2-25] 대륙이 'AS'인 국가들의 정보를 검색해 보자
df.loc[df['대륙']=='AS'].info()

- 참 거짓으로 나오지 않는 사칙연산 같은 건 loc 쓰지 않음

In [ ]:
# [2-26] 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량')를 추가합니다
# 주류소비량 = '맥주' + '증류주' + '와인'
df['주류소비량'] = (df['맥주'])+(df['증류주'])+(df['와인'])

# [2-27] 주류소비량2 = ['맥주', '증류주', '와인']에 대해 DataFrame.sum(axis=1) 함수 사용
df['주류소비량2'] = df[['맥주', '증류주', '와인']].sum(axis=1)

#### (4) 추가, 제거 및 병합
- 행 추가 : df.append(추가할 데이터프레임)
- 컬럼 추가1 : df['새로운칼럼명'] = 
- 특정 위치에 컬럼 추가2 : df.insert(위치, 컬럼, 값) 단, inplace 동작됨
- 열 제거1 : df = df.drop('열이름')    
- 열 제거2 : df = df.drop(rows=['열이름1', '열이름2' ...])
- 컬럼 제거1 : df = df.drop('칼럼명', axis=1)
- 컬럼 제거2 : df = df.drop(columns=[컬럼명...])
- 컬럼 제거3 : del df['제거할칼럼명'] 단, inplce 동작됨
- 데이터프레임 행/열 전환 : df.T

In [ ]:
# [2-26] 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량')를 추가합니다
# 주류소비량 = '맥주' + '증류주' + '와인'
df['주류소비량'] = (df['맥주'])+(df['증류주'])+(df['와인'])

# [2-40] 세계의 각 컬럼별 평균을 구하여 DataFrame으로 만들고,
# worldwide라는 이름을 지정합니다
# 세계의 각 컬럼별 평균은 DataFrame.mean()을 사용합니다.
worldwide = pd.DataFrame(df.mean(axis=0))

# [2-41] worldwide의 행과 열을 전환해 wwT로 저장합니다.
wwT = worldwide.T
wwT

# [2-42] wwT의 맨 앞에 '국가' 컬럼을 'World Wide' 값으로 추가합니다.
# 여러 번 추가하면 안됨
wwT.insert(0,'국가','World Wide') # 값을 여러개 줄 땐 리스트로 묶어줘야 한다.
wwT

- 행 병합 : concat([합칠행1, 합칠행2] ..., axis=0)
- 열 병합 : concat([합칠열1, 합칠열2] ..., axis=1)
* df.append()와 달리 여러 개의 df를 목록을 주어 한 번에 합칠 수 있음

In [ ]:
# [2-44] wwT와 korea 를 합쳐 하나의 DataFrame을 생성하여 df2로 저장합니다.
df2 = pd.concat([wwT,korea], axis=0)
df2

# [3-5] df2016, df2017, df2018, df2019를 합쳐 한 개의 DataFrame으로 만들어 df라는 이름을 지정합니다.
dfList = [df2016, df2017, df2018, df2019]
df = pd.concat(dfList, axis=0)

# concat을 하더라도 인덱스 번호는 유지되기 때문에 인덱스번호 다시 설정해줘야함
df.index = pd.RangeIndex(len(df))
# concat 할때부터 옵션으로 인덱스 잡아주기
df = pd.concat(dfList, ignore_index=True, axis=0)


### 6. 시각화

In [ ]:
# [3-25] df1의 '이산화질소농도(ppm)':'일산화탄소농도(ppm)'의 결측치의 상태를 그래프로 확인해 봅니다
# seaborn.heatmap()를 사용하며, df.isna()를 데이터로 지정해 확인할 수 있습니다.
# 흐린색이 결측치를 의미합니다.
import koreanize_matplotlib

plt.figure(figsize=(14, 10))    # 그래프 크기
ax = sns.heatmap(df1.loc[:, '이산화질소농도(ppm)':'일산화탄소농도(ppm)'].isna(), cbar=False)    # 히트맵으로
ax.set_xticklabels(ax.get_xticklabels(),fontsize=13, rotation=0)    # 눈금
plt.show()

    4. 범주형 데이터 라벨링
        1) LabelEncoding
        2) one-hot-encoding(getdummys()?)
    5. 뉴메리컬 데이터 스케일링
        1) StandardScaler
        2) MinMaxScaler
        
    6. 모델링
        1) 지도학습 분류 : RandomforestClassifier
        2) 지도학습 회귀 : RandomforestRegressor
        3) 비지도학습 군집화
    7. 모델 적용
        1) train_test_split(층화 추출 옵션stratify = y)
        2) new - fit - predict
    8. 모델 성능 확인 : 문제에서 요구하는 평가지표에 맞게 수행
        1) 지도 학습
            - 정확도(accuracy)
            - 정밀도(precision)
            - 재현율(recall)
            - 특이도(Specificity)
            - f1_score : 
            - ROC(Receiver Operating Characteristic) 곡선: 
            - AUC(Are Under Curve) : 
        2) 회귀 학습
            - SSE(Sum Squared Error) : 
            - MSE(Mean Squared Error) : 
            - RMSE(Root Mean Squared Error) :
            - MAE(Mean Absolute Error) : 
            - 결정계수(R2) :
            - Adjusted R2 : 
            - MSPE(Mean Squared Percentage Error) : 
            - MAPE(Mean Absolute Percentage Error) :
            - RMSLE(Root Mean Squared Logarithmic Error) : 
            - AIC(Akaike Information Criterion) : 
            - BIC(Bayes Information Criteria) : 
        3) 비지도 학습
            - 실루엣 계수(Silhouette) : 
            - Dunn lndex : 
            - 군집 분석
            - 연관 분석
    9. 예측 값 저장 
        submission.columns([''])
        to_csv('.csv', index=False)



유형3 : 데이터 검증 2문제(15*2)
    1. 데이터 불러오기 : pd.read_csv('.csv', index_col=0)
    2. 데이터 파악하기 : data.describe()
    3. 데이터 검증하기
    귀무가설(h0) 대신 대립가설(h1)을 선택할 시 오류를 범할 최대의 확률 값으로 0.05 이하이면 유의하여 대립가설 채택, 일반적으로 채택/기각은 귀무가설 기준
        01) t-검정
            (1)
            (2)
            (3)
        02) 정규성 검정
            (1)
            (2)
        03) One-way ANOVA
        04) Two-way ANOVA
        05) 카이제곱 검정
            (1) 독립성 검정
            (2) 동질성 검정
            (3) 적합성 검정
        06) 순위 검정
            (1) 윌콕슨 순위합 검정
            (2) 크루스칼 - 왈리스 검정
        07) 부호검정
            (1) 부호검정
            (2) 맥네마 검정
        08) 다변량 추론
            (1) 다중 회귀 분석

    4. 결과값 입력하기



t-검정 이해하기
